In [1]:
import requests
import os
import shutil
import sqlite3
import pandas as pd
from time import sleep

# Code for grabbing our sqlite file off the internet
global dump

"""
Used to download the Chinook database.
"""
def download_file():
    global dump
    url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
    dump = requests.get(url, stream=True).raw

'''
Used to save the downloaded Chinook Database into an sqlite database file.
'''
def save_file():
    global dump
    location = os.path.relpath("exampledb.sqlite")
    with open("exampledb.sqlite", 'wb') as location:
        shutil.copyfileobj(dump, location)
    del dump

In [2]:
"""
This code snippet downloads the Chinook database, connects to it, and prepares for queries to be executed.
"""
# Grabbing copy of database
download_file()
# Saving copy of database to a local file
save_file()
# Create a connection object that represents a database
conn = sqlite3.connect("exampledb.sqlite")
# Once the connection to the database is opened, we create a Cursor object to execute queries
c = conn.cursor()

In [3]:
"""
Write a Query that returns the following data from the Customer table,
Only return customers from Canada or the United States whose last name starts with the letter M.
"""
c.execute("""
SELECT FirstName || ' ' || LastName AS FullName, Company, City, State
FROM customer
WHERE (Country = 'Canada' OR Country = 'USA') AND LastName LIKE 'M%'
ORDER BY City;
""")

c.fetchall()

[('Dan Miller', None, 'Mountain View', 'CA'),
 ('Aaron Mitchell', None, 'Winnipeg', 'MB')]

In [4]:
"""
Write a query which returns the following information from the Artist, Album and Track tables.
Only returns tracks that have the word “dancing” somewhere in the track name.
"""
c.execute("""
SELECT a.Name AS ArtistName, al.Title AS AlbumTitle, t.Name AS TrackName
FROM artist a
INNER JOIN album al ON a.ArtistId = al.ArtistId
INNER JOIN track t ON al.AlbumId = t.AlbumId
WHERE t.Name LIKE '%dancing%'
ORDER BY a.Name;
""")

c.fetchall()

[('Led Zeppelin', 'Houses Of The Holy', 'Dancing Days'),
 ('U2', 'B-Sides 1980-1990', 'Dancing Barefoot'),
 ('Van Halen', 'Diver Down', 'Dancing In The Street')]

In [5]:
"""
Create a query that returns two columns one with the employee’s first and last name
and the second with their managers first and last name. Notice that General Manager
Andrew Adams should report to himself.
"""
c.execute("""
SELECT e.FirstName || ' ' || e.LastName AS Employee,
CASE WHEN e.EmployeeId = 1 THEN 'Andrew Adams reports to himself'
ELSE m.FirstName || ' ' || m.LastName END AS Manager
FROM employee AS e
LEFT JOIN employee AS m
ON e.ReportsTo = m.EmployeeId;
""")

c.fetchall()

[('Andrew Adams', 'Andrew Adams reports to himself'),
 ('Nancy Edwards', 'Andrew Adams'),
 ('Jane Peacock', 'Nancy Edwards'),
 ('Margaret Park', 'Nancy Edwards'),
 ('Steve Johnson', 'Nancy Edwards'),
 ('Michael Mitchell', 'Andrew Adams'),
 ('Robert King', 'Michael Mitchell'),
 ('Laura Callahan', 'Michael Mitchell')]

In [6]:
"""
Report only shows albums that have more than ten tracks.
"""
c.execute("""
SELECT al.Title AS AlbumTitle, COUNT(t.TrackId) AS NumberOfTracks
FROM album AS al
LEFT JOIN track AS t
ON al.AlbumId = t.AlbumId
GROUP BY al.AlbumId, al.Title
HAVING COUNT(t.TrackId) > 10
ORDER BY AlbumTitle;
""")

c.fetchall()

[('20th Century Masters - The Millennium Collection: The Best of Scorpions',
  12),
 ('A Matter of Life and Death', 11),
 ('A Real Dead One', 12),
 ('A Real Live One', 11),
 ('A TempestadeTempestade Ou O Livro Dos Dias', 15),
 ('A-Sides', 17),
 ('Ace Of Spades', 15),
 ('Achtung Baby', 12),
 ('Acústico', 22),
 ('Acústico MTV', 21),
 ('Acústico MTV [Live]', 17),
 ('Afrociberdelia', 23),
 ('Album Of The Year', 12),
 ('Alcohol Fueled Brewtality Live! [Disc 1]', 13),
 ("All That You Can't Leave Behind", 11),
 ('American Idiot', 13),
 ('Angel Dust', 14),
 ('Ao Vivo [IMPORT]', 19),
 ('Appetite for Destruction', 12),
 ('Are You Experienced?', 17),
 ('Arquivo II', 12),
 ('Arquivo Os Paralamas Do Sucesso', 16),
 ('As Canções de Eu Tu Eles', 14),
 ('Audioslave', 14),
 ('Axé Bahia 2001', 14),
 ('B-Sides 1980-1990', 15),
 ('BBC Sessions [Disc 1] [Live]', 14),
 ('Back to Black', 12),
 ('BackBeat Soundtrack', 12),
 ('Barulhinho Bom', 18),
 ('Battlestar Galactica (Classic), Season 1', 24),
 ('Battlest

In [7]:
conn.commit()
## To close connections to the database
conn.close()